In [ ]:
from backend.roughplanning.RoughPlanning import RoughPlanning
from backend.roughplanning.RoughPlanDrawer import RoughPlanDrawer
from backend.roughplanning.GNSS import GNSS_Point
import numpy as np

number_of_lines = 4
number_of_segments = 5
line_length = 300
min_elevation = 10 # gon

azimuths = [400 / number_of_lines * i for i in range(number_of_lines)]
elevation_angles = []

pt = GNSS_Point(name="3001", easting= 1_643_504, northing=1_260_541, floor_height=424.5)

rough1 = RoughPlanning(point=pt, dem_path="./testdata/raster.tif", method='CONVENTIONAL')
lines = rough1.create_lines(number_of_lines=number_of_lines, line_length=line_length)

for line in lines: # may  later be seperated on many cores
    segments = rough1.segment_line(number_of_segments=number_of_segments, line=line) # segment per line
    rough1.transform_linesegments(line_points=segments) # transformed segment per line
    profile = rough1.create_profile(line_points=segments) # create profile-Object
    max_alpha = rough1.get_max_angle(profile=profile)
    elevation_angles.append(max_alpha * 200 / np.pi)

drawer1 = RoughPlanDrawer()
drawer1.draw_panorama_diagram(azimuths=azimuths, elevation_angles=elevation_angles, min_elevation=min_elevation, image_path=f"./panorama_{pt.name}.png", pointname=pt.name)
drawer1.draw_polar_diagram(azimuths=azimuths, elevation_angles=elevation_angles, min_elevation=min_elevation, image_path=f"./polar_{pt.name}.png", pointname=pt.name)
drawer1.save_legend(legend_path="./legend.png")



In [1]:
from backend.roughplanning.RoughPlanning import RoughPlanning
from backend.roughplanning.RoughPlanDrawer import RoughPlanDrawer
from backend.roughplanning.GNSS import GNSS_Point
from backend.roughplanning.PDFCreator import PDFCreator



pt = GNSS_Point(name="3001", easting= 1_643_504, northing=1_260_541, floor_height=424.5)
pt2 = GNSS_Point(name="3002", easting= 1_643_504, northing=1_260_541, floor_height=424.5)

pdfcreator = PDFCreator(results_path="./bla.pdf")

pdfcreator.create_protocol(points=[pt, pt2], projectname="bal", projectleader="ich")

FileNotFoundError: Die Datei wurde nicht gefunden!